In [3]:
import os,sys
sys.path.append(os.path.abspath('./../../'))
from Controllers.MarkedCellController import MarkedCellController
from Controllers.SqlController import SqlController
from model.annotation_points import MarkedCellView
from sqlalchemy import inspect
inst = inspect(MarkedCellView)
attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
import pandas as pd
from cell_extractor.DetectorMetrics import AnnotationProximityTool
import numpy as np
from collections import Counter
user_look_up = {38:'Marissa',41:'Julian'}

def get_DataFrame_from_query_result(results,category,factor):
    values = []
    for i in results:
        source = i.source.value
        if '-' in source:
            source = source.split('-')[1]    
        x,y,z = np.array([i.x,i.y,i.z]).astype(float)/factor
        values.append([x,y,z,f'{category}_{user_look_up[i.FK_annotator_id]}_{source}'])
    # values = [[eval(f'j.{i}')  for j in results ] for i in attr_names]
    df = pd.DataFrame(dict(zip(['x','y','section','name'],np.array(values).T)))
    df["x"] = pd.to_numeric(df["x"])
    df["y"] = pd.to_numeric(df["y"])
    df["section"] = pd.to_numeric(df["section"])
    return df

In [4]:
animal = 'DK41'
controller = SqlController()
factor = controller.get_resolution(animal)
controller = MarkedCellController()
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':11}
round3_sures = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'Sure',factor)
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':12}
round3_unsures = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'Unsure',factor)
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':13}
connected = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'Connected',factor)
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':14}
false_negatives = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'False_negative',factor)


KeyError: 'x'

In [85]:
tool = AnnotationProximityTool()
tool.pair_distance=0.1
tool.set_annotations_to_compare(round3_sures)
tool.find_equivalent_points()
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 1000
after removing duplicates 500


[(('Sure_Marissa_POSITIVE', 'Sure_Julian_POSITIVE'), 196),
 (('Sure_Marissa_NULL', 'Sure_Julian_NEGATIVE'), 106),
 (('Sure_Marissa_NULL', 'Sure_Julian_POSITIVE'), 94),
 (('Sure_Marissa_NEGATIVE', 'Sure_Julian_NEGATIVE'), 48),
 (('Sure_Marissa_POSITIVE', 'Sure_Julian_NEGATIVE'), 47),
 (('Sure_Marissa_NEGATIVE', 'Sure_Julian_POSITIVE'), 9)]

In [86]:
tool = AnnotationProximityTool()
tool.pair_distance=0.1
tool.set_annotations_to_compare(round3_unsures)
tool.find_equivalent_points()
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 8982
after removing duplicates 4491


[(('Unsure_Marissa_NULL', 'Unsure_Julian_NULL'), 3135),
 (('Unsure_Marissa_NEGATIVE', 'Unsure_Julian_NEGATIVE'), 1011),
 (('Unsure_Marissa_POSITIVE', 'Unsure_Julian_POSITIVE'), 266),
 (('Unsure_Marissa_POSITIVE', 'Unsure_Julian_NEGATIVE'), 60),
 (('Unsure_Marissa_NEGATIVE', 'Unsure_Julian_NULL'), 12),
 (('Unsure_Marissa_POSITIVE', 'Unsure_Julian_NULL'), 4),
 (('Unsure_Marissa_NEGATIVE', 'Unsure_Julian_POSITIVE'), 3)]

In [90]:
tool = AnnotationProximityTool()
tool.pair_distance=1
tool.set_annotations_to_compare(connected)
tool.find_equivalent_points()
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 43
after removing duplicates 43


[(('Connected_Marissa_POSITIVE',), 43)]

In [88]:
tool = AnnotationProximityTool()
tool.pair_distance=5
tool.set_annotations_to_compare(false_negatives)
tool.find_equivalent_points()
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 917
after removing duplicates 778


[(('False_negative_Marissa_POSITIVE',), 385),
 (('False_negative_Julian_POSITIVE',), 254),
 (('False_negative_Julian_POSITIVE', 'False_negative_Marissa_POSITIVE'), 139)]